In [2]:
import json
import itertools
import xgi
import numpy as np
import networkx as nx
from tqdm import tqdm
import pickle as pk
from itertools import compress

dataset='congress-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-bills_simplices';name_file='{0}'.format(dataset);
#dataset='senate-committees_simplices';name_file='{0}'.format(dataset);
#dataset='house-committees_simplices';name_file='{0}'.format(dataset);
#dataset='email-Enron_simplices'; name_file='{0}'.format(dataset); 
#dataset='email-Eu_simplices'; name_file='{0}'.format(dataset); 
#dataset='hyperedges-cat-edge-algebra-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-geometry-questions_simplices'; name_file='{0}'.format(dataset);
#dataset='hyperedges-cat-edge-music-blues-reviews'; name_file='{0}'.format(dataset);
#dataset='Mid1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Elem1'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='InVS15'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='SFHH'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LH10'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='LyonSchool'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='Thiers13'; name_file='aggr_15min_cliques_thr1_{0}'.format(dataset);
#dataset='M_PL_015_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_062_ECO_ins';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);
#dataset='M_PL_015_ECO_pl';name_file='{0}'.format(dataset);

with open('Data\{0}.json'.format(name_file)) as json_file:
    data = json.load(json_file)
    
data = [data[i] for i in range(len(data)) if len(data[i])>1] #select only interactions of size>=2
for i in range(len(data)): 
    data[i].sort() #order the interactions according to the nodes IDs, e.g. [2,1] -> [1,2]
data.sort() 
data = list(data for data,_ in itertools.groupby(data)) #delete fully coincident hyperedges
data.sort(key = len)
L=len(data)
size_max=len(data[L-1])

size=list([len(data[j]) for j in range(L)])

X = xgi.Hypergraph(data) #build the hypergraph
IDX = list(X.nodes)
IDX_e = list(X.edges) 
N=len(IDX)

M=range(2,size_max+1) 
k_step=1
K=range(0,1200,k_step) 

k_shell_dict={}
idx_orig = IDX

IDX_size = range(len(size))
k_max=np.zeros(len(M))

for j in idx_orig:
    k_shell_dict[j]=np.zeros(len(M))

for x in range(len(M)):
    m=M[x]  
    
    D=np.zeros(len(K))  

    idx_size = list(compress(IDX_size, np.greater_equal(size, m * np.ones(len(size))))) #consider only hyperedges of size >=m
    int_sel = list([data[i] for i in idx_size])
    X = xgi.Hypergraph(int_sel) #build hypergraph with only interactions of size >=m
    IDX = list(X.nodes) 
    IDX_e = list(X.edges) 

    for y in range(len(K)):
        kk = K[y]
        
        d_tot_m = np.zeros(len(IDX))
        prev_shell=IDX
            
        for i in range(len(IDX)):
            d_tot_m[i] = X.degree(IDX[i])
        
        idx_n_remove = list(compress(IDX, np.greater(kk * np.ones(len(d_tot_m)),d_tot_m))) #nodes with degree<k are removed
        X.remove_nodes_from(idx_n_remove)
        IDX = list(X.nodes)
        IDX_e = list(X.edges)
                
        sizes = [val for (edge, val) in X.edge_size()] #hyperedges with size <m are removed 
        idx_e_remove = [IDX_e[i] for i in range(len(IDX_e)) if sizes[i]<m] 
        X.remove_edges_from(idx_e_remove)
        
        int_sel = [list(e) for e in X._edge.values()] #fully coincident hyperedges are removed
        int_sel.sort() 
        int_sel = list(int_sel for int_sel, _ in itertools.groupby(int_sel))
        X = xgi.Hypergraph(int_sel) #build new hypergraph
        IDX = list(X.nodes)
        IDX_e = list(X.edges)

        while len(idx_n_remove)>0 or len(idx_e_remove)>0: 
            
            d_tot_m = np.zeros(len(IDX))  
            
            for i in range(len(IDX)):
                d_tot_m[i] = X.degree(IDX[i])
                
            idx_n_remove = list(compress(IDX, np.greater(kk * np.ones(len(d_tot_m)),d_tot_m))) #nodes with degree<k are removed
            X.remove_nodes_from(idx_n_remove)
            IDX = list(X.nodes)
            IDX_e = list(X.edges)

            sizes = [val for (edge, val) in X.edge_size()] #hyperedges with size <m are removed 
            idx_e_remove = [IDX_e[i] for i in range(len(IDX_e)) if sizes[i]<m]
            X.remove_edges_from(idx_e_remove)
            
            int_sel = [] #fully coincident hyperedges are removed
            int_sel = [list(e) for e in X._edge.values()]
            int_sel.sort()
            int_sel = list(int_sel for int_sel, _ in itertools.groupby(int_sel))
            X = xgi.Hypergraph(int_sel)
            IDX = list(X.nodes)
            IDX_e = list(X.edges)
            
        shell_kk = list(sorted(set(prev_shell) - set(IDX)))
        for j in shell_kk:
            k_shell_dict[j][x]=kk-k_step

        D[y] = len(X.nodes)
        if y > 0:
            if D[y] == 0 and D[y - 1] != 0:
                k_max[x] = kk - k_step #maximum connectivity at order m
        if D[y]==0:
            break #stop the decomposition when the (k,m)-core is empty
            
pk.dump(k_shell_dict, open('K-shell_dict_{0}_hyper.pck'.format(dataset),'wb')) #m-shell index

with open('K-max_{0}_hyper.json'.format(dataset),'w') as f:
    json.dump(k_max.tolist(), f) #maximum connectivity
    
R_dict={} #size-independent hypercoreness
for y in k_shell_dict:
    R_dict[y]=sum(np.array(k_shell_dict[y])/np.array(k_max))

pk.dump(R_dict, open('R_dict_{0}_hyper.pck'.format(dataset),'wb'))

Psi=[] #Psi(m)  distribution of hyperedges size
for m in range(2,size_max+1):
    Psi.append(size.count(m)/len(size))

R_w_dict={} #frequency-based hypercoreness
for y in k_shell_dict:
    R_w_dict[y]=sum(np.array(Psi)*np.array(k_shell_dict[y])/np.array(k_max))
    
pk.dump(R_w_dict, open('R_w_dict_{0}_hyper.pck'.format(dataset),'wb'))